# Crash Report March 8, 2017 Release
## First Recorded 3/9/17



Version: 2016100028

In [2]:
"""
Add module path to sys path -- necessary if running from jupyter notebook an not an IDE
"""

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
"""
General import statements
"""
import crash_analysis
import pandas as pd

In [ ]:
"""
Download crashes by date range
"""


In [2]:
"""
Parse zipfiles into dataframe
"""
from crash_analysis.parser import extract_zipfiles, xmldocs_to_dataframe
zipfile_location = 'C:\\CrashReportsMarch8\\'

extract_zipfiles(zipfile_location)
df = xmldocs_to_dataframe(zipfile_location)

df.drop_duplicates(inplace=True)

In [4]:
"""
Save/load dataframe to/from CSV
"""
from crash_analysis import read_csv

new_data = False
cache_location = 'src/data/201608003_march9_535pm.csv'

if new_data:
    df.to_csv(cache_location, encoding='utf-8')
else:
    df = read_csv(cache_location)

FileNotFoundError: [Errno 2] No such file or directory: 'src/data'

In [3]:
"""
Get metadata about dataframe
"""
col_names = crash_analysis.get_columns(df)
versions = df['AppVersion'].value_counts()
num_crashes = len(df)

# March 9, 2017: 452


print(col_names)
print(versions)
print(num_crashes)

['Unnamed: 0', 'ACCDT_Field', 'Active_ClientFileName', 'Active_Field', 'Active_Form', 'Active_FormsetID', 'Active_FormsetVersion', 'AppName', 'AppVersion', 'BasWin16.INI', 'Batch_ClientFileName', 'CrashGUID', 'CrashRpt', 'Current_Calcsection', 'CustNum', 'CustomProps', 'DataFileCount', 'ExceptionAddress', 'ExceptionCode', 'ExceptionModule', 'ExceptionModuleBase', 'ExceptionModuleVersion', 'ExceptionType', 'FileList', 'FormsPrinter', 'GUIResourceCount', 'GeoLocation', 'ImageName', 'InnerException', 'InstallType', 'Last_Calcsection', 'ManagedException', 'ManagedException.txt', 'MemoryUsageKbytes', 'Message', 'OSIs64Bit', 'OpenHandleCount', 'OperatingSystem', 'ProWin16.INI', 'ProblemDescription', 'Source', 'StackTrace', 'SystemTimeUTC', 'TimeStamp', 'WorkStationName', 'WorkStationType', 'crashdump.dmp', 'crashrpt.xml']
2016100028    452
Name: AppVersion, dtype: int64
452


In [8]:
"""
Customer Description Analysis
"""
from crash_analysis.analysis import remove_empty, get_column, stem_frequency
customer_desc = remove_empty(get_column(df, 'ProblemDescription'))

stem_frequency(customer_desc)

print

In [6]:
"""
Frequency of error messages and stack traces
"""

from crash_analysis import filter_dataframe


def get_value_from_hist(n, field='Message'):
    field_hist = df[field].value_counts()
    assert n < len(field_hist)
    return field_hist.keys()[n], field_hist.values[n]

df_filt = None

def print_parings(n, field1='Message', groupings=['AppName', 'InstallType', 'OperatingSystem'], print_output=True):
    top_key, count = get_value_from_hist(n, field1)
    df_filtered = filter_dataframe(df, **{field1: top_key})
    global df_filt
    df_filt = df_filtered.copy()
    if type(groupings) is type(list()):
        field2_hist = df_filtered.groupby(groupings)['CustNum'].nunique()
    else:
        field2_hist = df_filtered[groupings].value_counts()

    if print_output:
        print("Query {0} \t count: {1} \t {2}: {3}".format(n+1, count, field1, top_key.encode('ascii', 'ignore')))
        print(field2_hist)
        print(sum(field2_hist))
        print '\n'
    
#

In [68]:
df['ExceptionAddress'].str.endswith('ed0a').value_counts()

def parse_exception_address(exad_str):
    exad_str = exad_str[::-1] # reverse string

    retstr = ''

    if(exad_str[:4] == 'a0de'):
        retstr += 'A'


    else:
        retstr += 'B'

    return retstr

ucrt_df = filter_dataframe(df, ExceptionModule='C:\Windows\System32\ucrtbase.dll')
rev_excep_addr_df = ucrt_df['ExceptionAddress'].apply(parse_exception_address)
# rev_excep_addr_df.srt.startswith('')
rev_excep_addr_df.value_counts()
# print(len(ucrt_df))

height has been deprecated.

Query 1 	 count: 811 	 ExceptionModule: C:\Windows\System32\ucrtbase.dll
CustNum     CrashGUID                             ProblemDescription                                                                                                
0000008012  39f728a9-b5ac-45f8-b736-7e46c92d0ab2  Saving a new client file                                                                                              1
0000132119  0825ec0a-1d82-44e7-bade-e16d140a1e9c  Crash#6 today.  was saving a tax file                                                                                 1
            19e60d34-9d25-4167-bc9d-e2dc561bd428  CRASH #8 TODAY.  WAS ENTERING INFORMATION INTO FORM                                                                   1
            1ba3271a-9a4d-4b42-89ff-d33be47f82d1  Crash #1.  Was saving a tax file                                                                                      1
            30b98404-a954-400e-b58b-1c711db5e6e6  sav

[None]

In [31]:
# pd.set_option('display.max_rows', 5000)
# pd.set_option('display.height', 5000)



# [print_parings(n, field1='ExceptionAddress', groupings='CrashGUID') for n in [6]]

# df_filt[df_filt['ExceptionAddress'].str.contains('0xffffffffffffec6c')]
df[df['ExceptionAddress'] == '0xffffffffffffec6c'].groupby(['CustNum'])['CrashGUID'].count()
# df[df['CustNum'].str.contains('213368').fillna(value=False)]['CustNum'].value_counts()[:10]

# df[df['CustNum'].str.contains('213368').fillna(value=False)]['ExceptionAddress']

# df[df['CustNum']=='0955920737']['OperatingSystem']

CustNum
7743220085    1
7770129748    1
Name: CrashGUID, dtype: int64

In [30]:
# str_to_hex = lambda x: hex(int(x, 16)) if isinstance(x, str) else int(x) if x is NaN) else 0
# 
# df['ActualAddress'] = df['ExceptionAddress'].apply(str_to_hex) - df['ExceptionModuleBase'].apply(str_to_hex)

# str_to_hex(0x1212) - str_to_hex(0x1010)

ValueError: cannot convert float NaN to integer

In [79]:
""" Customer Descriptions """

from crash_analysis import remove_empty
# print(one_cust.head())
pd.set_option('display.max_colwidth', 250)
print(sum(remove_empty(df['ProblemDescription']).value_counts()))

35


In [80]:
"""
Associate by keyterm
"""
from crash_analysis.analysis import associate_by_keyterms

associate_by_keyterms(df, 'ProblemDescription', 'Message', min_count=2)
print

Message by Keyterm
keyterm: program
Method not found: 'Void Intuit.Application.ProSeries.OptionSetupWizardAPI.DataModels.AdditionalInfo..ctor(System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String, System.String)'.    2
Method not found: 'Int32 System.Runtime.InteropServices.Marshal.SizeOf(!!0)'.                                                                                                                                                                                                                      1
External component has thrown an exception.                                                                                                                                                                                                                                                        1
Name: Message, dtype: int64

keyterm: open
External component has thrown an exception